In [1]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import lib
import pandas as pd
import numpy as np
from scipy.ndimage.interpolation import shift
from scipy.stats import t, shapiro, gaussian_kde
from scipy.optimize import minimize, fsolve

# Read in data from CSV files
prices = pd.read_csv("DailyPrices.csv", parse_dates=[0], index_col=0)
portfolios = pd.read_csv("portfolio.csv")

# Calculate returns of the prices using pd_return_calculate function
returns = lib.pd_return_calculate(prices)


# Group the portfolios dataframe by 'Stock' and sum the 'Holding' column
total_holdings = portfolios.groupby('Stock')['Holding'].sum().reset_index()

# Add a new 'Portfolio' column with value 'Total' to the total_holdings dataframe
total_holdings['Portfolio'] = 'Total'

# Concatenate the original portfolios dataframe and the total_holdings dataframe
portfolios = pd.concat([portfolios, total_holdings])



/tmp/ipykernel_955/1638063690.py:8: DeprecationWarning: Please use `shift` from the `scipy.ndimage` namespace, the `scipy.ndimage.interpolation` namespace is deprecated.
  from scipy.ndimage.interpolation import shift


In [2]:
t_params = {}
hist_data = []
sim_data = []

# Loop over each column in the returns dataframe
for col in returns:
    # Get the stock returns for the current column and standardize them
    stock_returns = returns[col] - returns[col].mean()
    # Add the standardized returns to the historical data list
    hist_data.append(stock_returns)

    # Fit a Generalized t distribution to the stock returns
    df, loc, scale = t.fit(stock_returns, method='MLE')
    t_params[col] = [df, loc, scale]

    # Simulate based on the t distribution for the current stock
    sim_data.append(t(df, loc, scale).rvs(10000))


hist_data = np.array(hist_data)
sim_data = np.array(sim_data)


# create pandas dataframe for the returns
historical_returns = pd.DataFrame(columns=returns.columns, data=hist_data.T)

In [3]:
# for historical data
current_prices = pd.DataFrame({"Price":prices.iloc[-1]})

# Loop over each portfolio in the groupby object
for portfolio_index, portfolio in portfolios.groupby('Portfolio'):
    # Set the index to 'Stock' and join with current_prices dataframe
    portfolio = portfolio.set_index('Stock')
    portfolio = portfolio.join(current_prices.loc[portfolio.index])

    # Calculate simulated returns and values
    sim_prices_change = historical_returns[portfolio.index] * portfolio['Price'].T
    sim_values_change = sim_prices_change @ portfolio['Holding']

    # Calculate VaR and ES using the lib module
    historic_var = lib.calculate_var(sim_values_change)
    historic_es = lib.calculate_es(sim_values_change)

    # Print results
    print(f"Portfolio {portfolio_index} VaR: {historic_var}")
    print(f"Portfolio {portfolio_index} ES: {historic_es}")
    print()


     

Portfolio A VaR: 8805.507754785509
Portfolio A ES: 10438.09016614881

Portfolio B VaR: 6981.307577790557
Portfolio B ES: 8945.79531744502

Portfolio C VaR: 5496.294533012176
Portfolio C ES: 7436.626667146727

Portfolio Total VaR: 21076.418322771402
Portfolio Total ES: 26687.791306205952

